In [ ]:
import torch
from embed_llm.models.augmented_model import EmbedAugPipeline

/home/hippolytepilchen/micromamba/envs/llm_embed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/Llama3.2-3B'
# llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/Gemma7B'
llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/mistral_7B'
ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/act1a73c20b533bed0e3328/checkpoints/checkpoint_001000' # Mistral 
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/act0a84d43f52dc15053eff/checkpoints/checkpoint_001000'# Llama
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/act7d8dde7a6309f5ca22de/checkpoints/checkpoint_001000' # Gemma

model_name = 'Mistral7B' # Mistral7B, Llama3.2-3B, Gemma7B
embed_model_name = 'NVEmbed'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
pipeline: EmbedAugPipeline = EmbedAugPipeline.load_inference_model(llm_path = llm_path, 
                                                                   ckpt_path = ckpt_path, 
                                                                   model_name = model_name, 
                                                                   embed_model_name = embed_model_name,
                                                                   device = device)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.01s/it]


In [ ]:
# 1 information in the doc which enables to answer the question
# 2 Same
# 3 Hard negative passage
# 4 Same



prompt_prefix = "Query: who wrote the song photograph by ringo starr\nAnswer: Ringo Starr\n\nQuery: who is playing the halftime show at super bowl 2016\nAnswer: Coldplay\n\nQuery: where was the world economic forum held this year\nAnswer: Davos\n\nQuery: where are the giant redwoods located in california\nAnswer: Humboldt County\n\nQuery: who has made the most premier league appearances\nAnswer: Gareth Barry\n\nQuery: "
prompts = ['who has most followers on instagram in world','who did the united states win its independence from', 'locations for the film an englishman who went up a hill', 'who is the valley of the dolls based on']
final_prompts = [prompt_prefix + prompt + '\nAnswer:' for prompt in prompts]

text_conditioning = ["This list contains the top 50 accounts with the most followers on the photo and video-sharing social platform Instagram. As of July 2019, the most followed user is Instagram's own account, with over 308 million followers. Cristiano Ronaldo is the most followed individual, with over 177 million followers. Fifteen accounts have exceeded 100 million followers on the site.",
                     "During the American Revolution, the legal separation of the thirteen colonies from Great Britain in 1776 actually occurred on July 2, when the Second Continental Congress voted to approve a resolution of independence that had been proposed in June by Richard Henry Lee of Virginia declaring the United States independent from Great Britain's rule. After voting for independence, Congress turned its attention to the Declaration of Independence, a statement explaining this decision, which had been prepared by a Committee of Five, with Thomas Jefferson as its principal author. Congress debated and revised the wording of the Declaration, finally approving it two days later on July 4. A day earlier, John Adams had written to his wife Abigail",
                     'The village was a primary location for the making of the film \"The Englishman Who Went Up a Hill But Came Down a Mountain\", which starred Hugh Grant. The hilltop scenes were filmed on the Gyrn, the long hill that overlooks the village. It was also featured in \"Monk\'s Hood\", an episode of \"The Cadfael Chronicles\"',
                     'Valley of the Dolls is the first novel by American writer Jacqueline Susann. Published in 1966, the book was the biggest selling novel of its year. To date, it has sold more than 31 million copies, making it one of the best-selling works in publishing history.']

answers = ['Instagram','Great Britain',"Llansilin in Powys",["Judy Garland", "Carole Landis", "Dean Martin", "Ethel Merman"]]

w_embeds = True
max_tokens = 512
temperature = 0.8

In [ ]:
generated_sequence = pipeline.generate(prompts = final_prompts, 
                                       text_conditioning = text_conditioning, 
                                       temperature = temperature, 
                                       max_tokens = max_tokens, 
                                       w_embeds = w_embeds)

/home/hippolytepilchen/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/7604d305b621f14095a1aa23d351674c2859553a/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),


In [ ]:
generated_sequence

['Selena Gomez\n\n\n\n\n\n\n\n\n\n\n\n\n\nGarete\n\n\n\n\nGrette\n\n\nRetire\nRetiret\nRetiretiretiretiretiretireti\n\n\n\nRetirete\n\n\n\n\nRetiretiretireti\n\nRetireti\n\nRetireti\nRetireti\n\n\n\nRetireti\n\n\nRetiretireti\n\nRetireti\n\nRetireti\nRetiretireti\n\n\n\n\n\n\n\n\n\nRetireti\n\n\n\n\nRetiretyretireti\n\n\n\n\n\n\n\n\n\n\nRetireti\n\n\n\n\n\nRetireti\n\n\n\nRetenti\n\nRetireti\n\n\n\nRetetryretiretiretyretireti\n\n\n\n\n\nRetetry\n\n\nRetirety\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRet\n\n\n\nRetirety\n\n\nRetiretyneti\n\nRet\n\neti\n\n\nRetetry\n\n\n\n\n\n\nRetirety\n\n\nneatir\n00CatherineRetieret\n1504finter5\n%16tirnet0.retnRette_teiretiretret5reticity0-Retite5retinet\nhetretiretyiretefrettiRettiniertireteir)etyRetighteriret\nty004\nRet\nti\ninerTITy&net\ntif\ncitet0\nREty\n\nHtty?t\n\n\nretnetheter\ntvheight00\nTickett0t}}}00j�TREE0�icate0�„�️�atique00ۮۇ¨ۇ‰ۍ۟ti��TYےۋ۟ۇۇۙۮۋ۫ےےۇ۟ۉۚۙۇۦۿۘۃۗۙۏۮۇ۟ۏۙۿۋےۋې۫ےۃۥۙےۋےۋ۬ۇ�',
 "Great Britain\n\n\n\n\n\n\n\n\n\nDeclar\n\nDee\n